In [3]:
#get dataframe of tweet_id and user_id
import pandas
import csv

df = pandas.read_csv('gamergate_tweets.csv')
print df

                  tweet_id     user_id
0       524349343353294848  1464599533
1       524349351259566080  1435309448
2       524349355528945664    19932466
3       524349359601635329   940132086
4       524349361074212864  1945343251
5       524349365431709696  2772953185
6       524349369152458753  2683063328
7       524349374168453120   513001579
8       524349375829381120   162572003
9       524349376588562433  2826306403
10      524349379772026881    16641565
11      524349382523904000  2756873076
12      524349386201899008   486160284
13      524349392296615936    18746944
14      524349393584267264  2323141220
15      524349395152945152  2793171236
16      524349402928787457    18623405
17      524349402551697410   171848975
18      524349403109130240  2549103608
19      524349407542513664    36790442
20      524349411430658049  2835488103
21      524349412122697728  1043436650
22      524349414908108801  2837972170
23      524349420444602369    22655375
24      52434942129183539

In [35]:
#import sys
#sys.path.append("/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages")

from twython import Twython

#get first 1000 tweet_ids
CONSUMER_KEY = "48r56upQvsnTiSDfY4CNwIpCQ"
CONSUMER_SECRET = "VqZhFW7GqogdKXJnpuS0umVe211bFgoCdaQlRdRMayvrbdDh3k"
OAUTH_TOKEN = "18000528-Scl8a1pLinqP4KWtaa3f8EN0M1vqAGTyYXMtsLIGm"
OAUTH_TOKEN_SECRET = "NcIvLeqVbwwBegabvo7946FVIaDN0CAaJ7aS2aV7pZFzC"

twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

tweet = twitter.lookup_status(id=current)
start = 0
end = 100
tweet_list = []
while end <= 5000:
    id_list = df["tweet_id"].tolist()
    current = id_list[start:end]
    start += 100
    end += 100
    for t in tweet:
        each_tweet = []
        each_tweet.append(t["retweet_count"])
        each_tweet.append(t["favorite_count"])
        each_tweet.append(t["text"])
        tweet_list.append(each_tweet)

In [36]:
#put into df    
tweets_df = pandas.DataFrame(tweet_list, columns=["Retweets", "Favorites", "Text"])
print tweets_df

      Retweets  Favorites                                               Text
0            0          0  Does everything now have to have "-gate" on th...
1            3          0  RT @MahdDogg: When Pro #GamerGate answers the ...
2            0          0  What Fuels #GamerGate Anger And Outrage? Gende...
3            0          0  https://t.co/MTWhp8kqcH here my new YouTube vi...
4            0          1  READ CONCERN TROLL GOES AFTER THE WIZARDS http...
5            1          0  RT @ItsAustin_13: I liked a @YouTube video fro...
6            8          0  RT @ItalyGG: Very commendable. I left workplac...
7            0          0  #GamerGate Is The Future Of Troll Politics @Te...
8            4          0  RT @megaspacepanda: Does #GamerGate have any s...
9            0          0  #forbes What Fuels #GamerGate Anger And Outrag...
10           6          0  RT @subtleblend: It's Monday, #GamerGate. How ...
11           3          0  #GamerGate Makes the Left Uncomfortable Becaus...

In [37]:
#use lexicon to get polarity of each tweet

#read and put positive words into list
with open('positive-words.txt') as f:
    positive = f.read().splitlines()
#trim list    
positive = positive[35:]
positive = set(positive)

#read and put negative words into list
with open('negative-words.txt') as f:
    negative = f.read().splitlines()
#trim list    
negative = negative[35:]
#remove problematic "naive"
negative.remove(negative[3007])
negative = set(negative)
    
lexrating = []
poscount = 0
negcount = 0

x = 0

for tweet in tweets_df["Text"]:
    words = tweet.split()
    for word in words:
        if word in positive:
            poscount = poscount + 1
        elif word in negative:
            negcount = negcount + 1
            
    #get polarity of tweet
    if negcount == 0: ratio = poscount
    elif poscount == 0: ratio = negcount * -1
    else : 
        if poscount > negcount: ratio = poscount - negcount
        else: ratio = (negcount - poscount) * -1
    
    lexrating.append(ratio)

    poscount = 0
    negcount = 0
    x += 1
        
    #add column of lexrating reviews
    tweets_df["Polarity"] = pandas.Series(lexrating)
    
print tweets_df.sort(["Polarity"])
        


      Retweets  Favorites                                               Text  \
1065         1          0  RT @WolfOfHermes: I remember now why I stopped...   
2468       485          0  RT @CHSommers: Didn't the editorial director o...   
1618         8          0  RT @ItalyGG: Very commendable. I left workplac...   
2469         1          0  RT @WolfOfHermes: I remember now why I stopped...   
2242         8          0  RT @ItalyGG: Very commendable. I left workplac...   
578          8          0  RT @ItalyGG: Very commendable. I left workplac...   
1632         1          0  RT @elpinchejavi: "What can we do to stop #Gam...   
1636       485          0  RT @CHSommers: Didn't the editorial director o...   
1637         1          0  RT @WolfOfHermes: I remember now why I stopped...   
564          0          0  @stillgray aka "Little Hitler" says @MercedesB...   
2476       485          0  RT @CHSommers: Didn't the editorial director o...   
1644       485          0  RT @CHSommers

/Users/Chelsea/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:49: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
